
A string decryptor and config extractor of Gurcu stealer which is a DotNet(.NET) stealer.

Sample SHA-256 Hash: [fb33ecb0d51761b0bde6977c7ae7eec18fd4c326bd73f3f4fac32e9abfb18575](https://tria.ge/231005-1qgbmshe48/behavioral1)

In [ ]:


import clr  # Import pythonnet's Common Language Runtime interface
file_path = r"C:\Path\To\Your\file"  # Specify the path to your .NET assembly
clr.AddReference("dnlib")  # Add reference to dnlib.dll (but this needs to point to a real path or GAC-loaded assembly)

import dnlib  # Attempting to import dnlib as a Python module 
from System.Reflection import Assembly, BindingFlags, MethodInfo
from dnlib.DotNet import ModuleDef, ModuleDefMD
from dnlib.DotNet.Emit import OpCodes
from dnlib.DotNet.Writer import ModuleWriterOptions
from dnlib.DotNet import *  # Import everything from dnlib.DotNet namespace
from dnlib.DotNet.Emit import OpCodes  # Import IL OpCodes used to analyze method instructions

module = dnlib.DotNet.ModuleDefMD.Load(file_path)  # Load .NET assembly into dnlib

In [ ]:
def rc4_decr(A_0, A_1, A_2):
    text = ""
    num = 0
    array = list(range(256))

    # Generate the key for scheduling
    text2 = A_1 + "kek" + A_2 # Key is formed by concatenating A_1, "kek", and A_2

    # Key-Scheduling Algorithm (KSA)
    for j in range(256):
        num = (ord(text2[j % len(text2)]) + array[j] + num) % 256
        array[j], array[num] = array[num], array[j]

    # Pseudo-Random Generation Algorithm (PRGA)
    for k in range(len(A_0)):
        num3 = k % 256
        num = (array[num3] + num) % 256
        array[num3], array[num] = array[num], array[num3]  # Swap values
        keystream_byte = array[(array[num3] + array[num]) % 256]
        text += chr(ord(A_0[k]) ^ keystream_byte)  # XOR and append to result

    return text


import re
# Converts all occurrences of escaped hex sequences like \\xHH
# in the input string into Unicode escape sequences of the form \\u00HH.
def convert_to_unicode(s): 
    # Replace \\xHH with \u00HH using regex
    return str(re.sub(r'\\x([0-9a-fA-F]{2})', r'\\u00\1', s))

In [ ]:
decryptor_method_name = "Izmsjtakgluyjfduncjfgh" # Decryptor method name

# Loop through all types (classes, structs, etc.) in the module
for type in module.GetTypes():
    for method in type.Methods:
        if not method.HasBody:
            continue  # Skip methods without body or stop if already found

        instrs = method.Body.Instructions

        # Iterate through each instruction in the method
        for ptr in range(len(instrs)):
            instr = instrs[ptr]

            # Check if it's a method call
            if instr.OpCode == OpCodes.Call or instr.OpCode == OpCodes.Callvirt:
                operand = instr.Operand

                if operand is not None and hasattr(operand, "Name"):
                    if operand.Name == decryptor_method_name:

                        # Make sure we have at least 3 instructions before the call and the Call instruction
                        if ptr >= 4:
                            instr0 = instrs[ptr] # Call instruction
                            instr1 = instrs[ptr - 1] # Key 1
                            instr2 = instrs[ptr - 2] # Key 2
                            instr3 = instrs[ptr - 3] # Encrypted Data

                            # Check if all 3 are string ldstr instruction
                            if instr1.OpCode == OpCodes.Ldstr and instr2.OpCode == OpCodes.Ldstr and instr3.OpCode == OpCodes.Ldstr:
                                key1 = instr1.Operand
                                key2 = instr2.Operand
                                data = instr3.Operand
                                data = convert_to_unicode(data) # to unicade
                                decr_data = rc4_decr(data,key2,key1) # decrypt data
                                
                                # Configuration extractor
                                # # len(decr) > 7 to exclude http:// withoud IP:Port
                                if len(decr_data) > 7 and "http://" in decr_data and "127.0.0.1" not in decr_data and "www.w3.org" not in decr_data and "ip-api" not in decr_data:
                                    print("C2")
                                    print(decr_data)
                                
                                # Patch the instructions
                                # Replace the 3 ldstr instructions with NOP
                                instr1.OpCode = OpCodes.Nop
                                instr2.OpCode = OpCodes.Nop
                                instr3.OpCode = OpCodes.Nop
                                # Replace the call instruction with a new ldstr instruction and its operand is the decrypted data
                                instr0.OpCode = OpCodes.Ldstr
                                instr0.Operand = decr_data
                                
# Save the modified module
options = ModuleWriterOptions(module)
options.Logger = dnlib.DotNet.DummyLogger.NoThrowInstance

# Write cleaned module content
module.Write("out.bin", options)

C2
http://217.145.238.175:80
http://135.181.98.45:8888
http://65.108.226.108:8080
http://45.155.171.134:8080
https://164.90.185.9:443
http://206.189.109.146:80
http://141.94.175.31:8098
http://116.202.101.219:8080
http://129.159.134.19:8080
http://107.161.20.142:8080
http://3.142.76.113:80
https://192.99.196.191:443
http://104.184.140.41:9000
http://205.185.123.66:8080
http://139.99.123.53:9191
https://44.228.161.50:443
http://154.31.165.232:80
http://139.84.231.199:8080
http://103.244.151.46:8080
http://129.151.109.160:8080
http://168.138.211.88:8099
http://103.226.125.218:80
http://216.250.190.139:80
https://52.196.241.27:443
https://13.112.250.213:443
http://189.115.63.77:8080